# Analysis

### Index
0. [Imports](#imports)
1. [System parameters](#parameters)
2. [Water molecules and ions inside the region](#region)
3. [Hydrogen bonds](#hbonds)
4. [Paths](#paths)
5. [Water stability](#stability)

### Imports <a name="imports"></a>

In [ ]:
import sys
sys.path.append('/home/jorge/research/peptides/simulations/peptides')
from mdtools.core import *
from mdtools.analysis import *

### System parameters <a name="parameters"></a>

In [ ]:
# Fixed parameters
'''
Los tubos están colocados:
1 2
3 4
'''

traj_name = "4t10s_run01"
N_tubes = 4 # Number of nantubes in the system
N_rings = 10 # Number of peptides in a nanotube
N_res = 8 # Number of resiudes in a peptide

In [ ]:
# Get the reslist of the residues that are pointing towards the channel
channel_reslist = get_channel_reslist(N_rings, N_res, [0, 1, 2, 3], [[4, 4], [2, 6], [0, 0], [6, 2]])
mask = "resid "
for res in channel_reslist:
    mask += str(res + 1) + " "
print(mask)

In [ ]:
# ONLY ONE TIME
# recenter_traj_RMSD(traj_name, N_tubes, N_res)

In [ ]:
traj = md.load(traj_name+"_RMSD.nc", top=traj_name+".parm7")
traj.center_coordinates()
myselections = ["resname LYS and name NZ", "resname LYN and name NZ", "resname TYD and name OH"]
p = MyParams(traj, N_tubes, N_rings, N_res, myselections)
p.IONs = traj.top.select("element Cl")
p.IONs = traj.top.select("resname TFA and name OD1 OD2")

### Water molecules and ions inside the region <a name="region"></a>

In [ ]:
# Water molecules and ions nearby the bundle
get_indices(traj, p.WATs, p.IONs, p.CAs, p.N_rings, layer=0, delta=1.0,
            save=True, savefileWATs="iterWATs", savefileIONs="iterIONs")

In [ ]:
# Load water molecules and ions
iterWATs = np.load("iterWATs.npy", allow_pickle=True)
iterIONs = np.load("iterIONs.npy", allow_pickle=True)

In [ ]:
# Water molecules and ions inside the lumina

print("Analysing lumen 0")
get_indices(traj, iterWATs, iterIONs, [CA for CA in p.CAs if CA.tube==0], p.N_rings, layer=3, boundary=2, delta=0.0,
            preselected=True, save=True, savefileWATs="iterWATs_lumen0", savefileIONs="iterIONs_lumen0")

print("Analysing lumen 1")
get_indices(traj, iterWATs, iterIONs, [CA for CA in p.CAs if CA.tube==1], p.N_rings, layer=3, boundary=2, delta=0.0,
            preselected=True, save=True, savefileWATs="iterWATs_lumen1", savefileIONs="iterIONs_lumen1")

print("Analysing lumen 2")
get_indices(traj, iterWATs, iterIONs, [CA for CA in p.CAs if CA.tube==2], p.N_rings, layer=3, boundary=2, delta=0.0,
            preselected=True, save=True, savefileWATs="iterWATs_lumen2", savefileIONs="iterIONs_lumen2")

print("Analysing lumen 3")
get_indices(traj, iterWATs, iterIONs, [CA for CA in p.CAs if CA.tube==3], p.N_rings, layer=3, boundary=2, delta=0.0,
            preselected=True, save=True, savefileWATs="iterWATs_lumen3", savefileIONs="iterIONs_lumen3")

In [ ]:
# Water molecules and ions inside the channels

print("Analysing channel 0")
get_indices_xtal(traj, p.WATs, p.IONs, get_atoms_in_reslist(p.CAs, channel_reslist), p.N_rings,
                 save=True, savefileWATs="iterWATs_channel0", savefileIONs="iterIONs_channel0")

print("Analysing channel 1")
get_indices_xtal(traj, p.WATs, p.IONs, get_atoms_in_reslist(p.CAs, channel_reslist), p.N_rings,
                 offsets=[np.array([-0.5, 0.0, 0.0]), np.array([0.5, 0.0, 0.0])],
                 save=True, savefileWATs="iterWATs_channel1", savefileIONs="iterIONs_channel1")

print("Analysing channel 2")
get_indices_xtal(traj, p.WATs, p.IONs, get_atoms_in_reslist(p.CAs, channel_reslist), p.N_rings,
                 offsets=[np.array([0.0, -0.5, 0.0]), np.array([0.0, 0.5, 0.0])],
                 save=True, savefileWATs="iterWATs_channel2", savefileIONs="iterIONs_channel2")

print("Analysing channel 3")
get_indices_xtal(traj, p.WATs, p.IONs, get_atoms_in_reslist(p.CAs, channel_reslist), p.N_rings,
                 offsets=[np.array([-0.5, -0.5, 0.0]), np.array([-0.5, 0.5, 0.0]), np.array([0.5, 0.5, 0.0]), np.array([0.5, -0.5, 0.0])],
                 save=True, savefileWATs="iterWATs_channel3", savefileIONs="iterIONs_channel3")

In [ ]:
# Check
iterWATs = np.load("iterWATs_channel0.npy", allow_pickle=True)
WATs = iterWATs[-1]
mask = "same residue as index "
for index in WATs:
    mask += str(index) + " "
mask

### Hydrogen bonds <a name="hbonds"></a>

In [ ]:
print("Analysing channel 0")
channel0_reslist = get_channel_reslist(N_rings, N_res, [0, 1, 2, 3], [[4, 4], [2, 6], [0, 0], [6, 2]])
analyse(p, traj, "channel0", reslist=channel0_reslist, distance_cutoff=2.5, angle_cutoff=120, xtal=True)

print("Analysing channel 1")
channel1_reslist = get_channel_reslist(N_rings, N_res, [0, 1, 2, 3], [[2, 6], [4, 4], [6, 2], [0, 0]])
analyse(p, traj, "channel1", reslist=channel1_reslist, distance_cutoff=2.5, angle_cutoff=120, xtal=True)

print("Analysing channel 2")
channel2_reslist = get_channel_reslist(N_rings, N_res, [0, 1, 2, 3], [[6, 2], [0, 0], [2, 6], [4, 4]])
analyse(p, traj, "channel2", reslist=channel2_reslist, distance_cutoff=2.5, angle_cutoff=120, xtal=True)

print("Analysing channel 3")
channel3_reslist = get_channel_reslist(N_rings, N_res, [0, 1, 2, 3], [[0, 0], [6, 2], [4, 4], [2, 6]])
analyse(p, traj, "channel3", reslist=channel3_reslist, distance_cutoff=2.5, angle_cutoff=120, xtal=True)

In [ ]:
# Check
mask = "resid "
for res in channel3_reslist:
    mask += str(res + 1) + " "
mask

In [ ]:
print("Analysing channel 0")
detail_hbonds("channel0")

print("Analysing channel 1")
detail_hbonds("channel1")

print("Analysing channel 2")
detail_hbonds("channel2")

print("Analysing channel 3")
detail_hbonds("channel3")

### Paths <a name="paths"></a>

In [ ]:
print("Analysing channel 0")
search_longestpaths(traj, "channel0", xtal=True)

print("Analysing channel 1")
search_longestpaths(traj, "channel1", xtal=True)

print("Analysing channel 2")
search_longestpaths(traj, "channel2", xtal=True)

print("Analysing channel 3")
search_longestpaths(traj, "channel3", xtal=True)

### Water stability <a name="stability"></a>

In [ ]:
water_channel_stability(traj)